In [111]:
# import h5 reader
import h5py
import numpy as np
import pandas as pd
import time
from os import listdir
from os.path import isfile, join
import pickle
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib
import scipy.optimize as opt
from matplotlib.ticker import LinearLocator
from scipy.interpolate import griddata, interp1d
import emcee
from joblib import Parallel, delayed, effective_n_jobs
import imageio
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

work_path = "/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/2d/InverseUniExp/"
h5_path = work_path + "output/fault/"
print("Number of workers available: ", effective_n_jobs(-1))

Number of workers available:  36


# Gaussian predictor

In [3]:
def string(val):
    return "{:.16e}".format(val)

In [4]:
# Import Gaussian-regression related functions
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

# Pre-process the data
from sklearn import preprocessing

# function train_GP
class GP_predictor:
    # Constructor
    def __init__(self, 
                 input_set, 
                 observation_set, 
                 GPkernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-3, 1e3)), 
                 n_optimizers = 9):
        # Scale input data
        self.input_set = [list(x) for x in input_set]
        self.observation_set = [list(x) for x in observation_set]
        self.input_dimension = len(self.input_set[0])
        self.observation_dimension = len(self.observation_set[0])
        self.trainset_length = len(self.input_set)
        
        self.input_scaler = preprocessing.StandardScaler()
        self.input_scaler.fit(np.array(self.input_set))
        
        # Scale output data
        self.observation_scaler = preprocessing.StandardScaler()
        self.observation_scaler.fit(np.array(observation_set))
        
        # Fit Gaussian process
        self.GP = GaussianProcessRegressor(kernel = GPkernel, n_restarts_optimizer = n_optimizers)
        # self.GP = MyGPR(kernel = GPkernel, n_restarts_optimizer = n_optimizers, max_iter = max_iterations)
        self.GP.fit(self.input_scaler.transform(np.array(self.input_set)), 
                    self.observation_scaler.transform(np.array(self.observation_set)))
        
    # Predict on a new input set
    def predict(self, new_input_set):
        # Predict new observation
        new_observation = self.observation_scaler.inverse_transform(
            self.GP.predict(
                self.input_scaler.transform(np.array(list(new_input_set)).reshape([-1, self.input_dimension]))
            )
        )
        
        return new_observation
        
    

In [92]:
# Class RunABatch
class RunABatch:
    # Constructor
    def __init__(self, input_set, work_path, FourierTerms = 16, distanceAbove = 2e-3, 
                 nOfQueryPts = 10, obsFlag = 'fault', fourierFlag = False):
        # Initialize data paths and batch parameters
        self.work_path = work_path
        self.input_set = [tuple(i) for i in input_set]
        self.h5_path = work_path + "output/faultFiles/"
        self.frontsurf_path = work_path + "output/frontsurfFiles/"
        self.FourierTerms = FourierTerms
        self.distanceAbove = distanceAbove
        self.nOfQueryPts = nOfQueryPts
        self.fourierFlag = fourierFlag
        
        # Store all cases in the h5_path
        self.existingCasesFile = self.h5_path + "CaseList.csv"
        
        # Flag for whether cases have been run
        self.casesExcuted = False
        
        # Get existing cases
        self.getExistingCasesOfInputSet()
        
        # Get input_set_toRun
        self.input_set_toRun = list(set(self.input_set) - self.existingCases)
        
        # Run cases
        self.runCases(self.input_set_toRun)
        
        # Get obsevations
        # self.Observations = self.getObservations(self.input_set)
        if obsFlag == 'fault':
            self.Observations = self.getObservations(self.input_set, self.nOfQueryPts)
        elif obsFlag == 'front':
            self.Observations = self.getObservationsFrontSurf(self.input_set, self.nOfQueryPts, self.distanceAbove)
        else:
            self.Observations = self.getObservationsEveryWhere(self.input_set)
            
    # Inline function gets [A, B] list
    def getABDrsThetaFstar(self, fileName):
        A_idx = fileName.find('A')
        B_idx = fileName.find('B')
        Drs_idx = fileName.find('D')
        Theta_idx = fileName.find('t')
        Fstar_idx = fileName.find('f')
        slash_idx = fileName.find('-fault')

        # Change this part !! Before applying to new name convention
        # print("fileName: ", fileName)
        A = float(fileName[A_idx + 1 : B_idx - 1])
        B = float(fileName[B_idx + 1 : Drs_idx - 1])
        Drs = float(fileName[Drs_idx + 3 : Theta_idx - 1])
        Theta = float(fileName[Theta_idx + 5 : Fstar_idx - 1])
        Fstar = float(fileName[Fstar_idx + 5 : slash_idx])
        return (A, B, Drs, Theta, Fstar)
    
    
    # Function get_existing_files_set
    def getExistingCasesOfInputSet(self):
        # Get all .h5 file names as a list
        myPath = self.h5_path
        onlyFiles = [f for f in listdir(myPath) if (isfile(join(myPath, f)) and f[-8 : ] == 'fault.h5')]
        self.existingCases = set([self.getABDrsThetaFstar(f) for f in onlyFiles])
        
    # Function runCases
    def runCases(self, input_set, nThreads=24):
        # Start the timer
        st_time = time.time()
        
        # Edit the directories
        shell_pathR = self.work_path + "RunJobsJP.sh"
        shellRead = open(shell_pathR, 'r')
        list_of_lines = shellRead.readlines()
        shellRead.close()

        # Divide the input_set into nThreads subsets
        if nThreads > len(input_set):
            nRealThreads = len(input_set)
        else:
            nRealThreads = nThreads
        
        if nRealThreads > 0:
            # Split with Real thread numbers
            split_input_set = np.array_split(input_set, nRealThreads)
            split_threadNo = list(range(1, nRealThreads + 1))
            
#             # Print splits
#             for inputSet, tNo in zip(split_input_set, split_threadNo):
#                 print(inputSet, tNo)
            
            # Submit the jobs
            time_thread = Parallel(n_jobs=nRealThreads, backend='threading')(
                delayed(self.runCasesOneThread)(list_of_lines, inputSet, tNo) for inputSet, tNo in zip(split_input_set, split_threadNo)
            )
        else:
            time_thread = [0.]
        
        # Get running time
        time_cost = time.time() - st_time
        
        # Print running time
        # print("Wall clock time to run all inputs: ", time_cost)
        # print("Sum of all thread times: ", sum(time_thread))
        
        # Return from shell
        self.casesExcuted = True
        
        return
    
    # Function runCasesOneThread
    def runCasesOneThread(self, list_of_lines, input_set, threadNo):
        # Start timer
        st_time = time.time()
        
        # Deep copy the list
        write_lines = list_of_lines.copy()
        
        # Set output path
        shell_pathW = self.work_path + "RunJobsJP-" + str(threadNo) + ".sh"
        
        # Extract parameters
        AA = [string(ele[0]) for ele in input_set]
        BB = [string(ele[1]) for ele in input_set]
        DD = [string(ele[2]) for ele in input_set]
        TT = [string(ele[3]) for ele in input_set]
        FF = [string(ele[4]) for ele in input_set]
        
        # Change the lines
        write_lines[9] = "AA=" + str(tuple(AA)).replace(',', '').replace('\'', '') + "\n"
        write_lines[10] = "BB=" + str(tuple(BB)).replace(',', '').replace('\'', '') + "\n"
        write_lines[11] = "DD=" + str(tuple(DD)).replace(',', '').replace('\'', '') + "\n"
        write_lines[12] = "TT=" + str(tuple(TT)).replace(',', '').replace('\'', '') + "\n"
        write_lines[13] = "FF=" + str(tuple(FF)).replace(',', '').replace('\'', '') + "\n"
        write_lines[14] = "threadNo=" + str(threadNo) + "\n"
        
        # Write to file
        shellWrite = open(shell_pathW, 'w')
        shellWrite.writelines(write_lines)
        shellWrite.close()
        
        #Excute the job
        !source $shell_pathW
        
        # Return time
        return time.time() - st_time
    
    # Function getObservations for the input_set
    def getObservations(self, input_set, nOfQueryPts):
        # Initialize Observations
        Observations = []
        
        # Check if the cases have been excuted
        if self.casesExcuted == False:
            return Observations
        
        # Rotation matrix Q
        alpha = 29 / 180 * np.pi
        Q = np.array([[np.cos(alpha), np.sin(alpha)], [-np.sin(alpha), np.cos(alpha)]])
        
        # VS start
        VSstart = np.array([0.006354, 0.003522])
        
        # Loop through all Inputs
        for input_ele in input_set:
            # Open the file
            h5_file = self.h5_path + "A" + string(input_ele[0]) + "_B" + string(input_ele[1]) \
                      + "_DRS" + string(input_ele[2]) + "_theta" + string(input_ele[3]) + "_fStar" + \
                      string(input_ele[4]) + "-fault.h5"
                     
            f = h5py.File(h5_file, 'r')

            # Get time
            times = np.array(f['time']).reshape([-1])
            times = times - np.min(times)
            nOfTSteps = times.shape[0]
            
            # Get coordinates
            coords = np.array(f['geometry']['vertices']) - VSstart
            Qcoords = coords @ Q.transpose()
            XXs = Qcoords[:, 0]
            
            # Get Slip rates [time, nOfNodes, spaceDim]
            SlipRates = np.array(f['vertex_fields']['slip_rate'])
            SlipRatesMag = np.linalg.norm(SlipRates, ord = 2, axis = 2)
            nOfNodes = SlipRatesMag.shape[1]
            
            xqs = 1e-3 * np.linspace(0., 45., nOfQueryPts)
            
            # Store the slip rates
            slip_rate_func = interp1d(XXs, SlipRatesMag, kind = 'cubic')  # [nOfTSteps, nOfNodes]
            
            # Change slip rate x to [nOfNodes, nOfTSteps]
            slip_rate_x = slip_rate_func(xqs).transpose()
            
#             # DEBUG LINES
#             print('nOfTSteps: ', nOfTSteps)
#             print('nOfNodes: ', nOfNodes)
#             print('SlipRatesMag shape: ', SlipRatesMag.shape)
#             print('slip_rate_x shape: ', slip_rate_x.shape)
            
            # Get the observations
            if self.fourierFlag:
                # Find the Fourier coefficients
                FourierTerms = self.FourierTerms
                T = np.max(times)

                # Compute the Fourier terms
                Ks = np.array(range(FourierTerms))
                coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * times)
                VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in slip_rate_x], 0)

                # Compute the fourier coefficients
                # print('time.shape: ', time.shape)
                observation = np.trapz(VxcoskPiTt, x=times)
                # print("observation shape: ", observation.shape)
                # Append the result from this file
                Observations.append(observation)
            else:
                # Find nFourier terms of values, evenly spaced in [0, T]
                T = np.max(times)
                tts = np.linspace(0., T, self.FourierTerms)
                observation_func = interp1d(times, slip_rate_x)
                observation = observation_func(tts).reshape([-1])
                Observations.append(observation)
                
        Observations = np.array(Observations)
        return Observations
    
    # Function getObservationsFrontSurf for the input_set
    def getObservationsFrontSurf(self, input_set, nOfQueryPts, distanceAbove):
        # Initialize Observations
        Observations = []
        
        # Rotation matrix Q
        alpha = 29 / 180 * np.pi
        Q = np.array([[np.cos(alpha), np.sin(alpha)], [-np.sin(alpha), np.cos(alpha)]])
        
        # VS start
        VSstart = np.array([0.006354, 0.003522])
        
        # Query points
        distance_above = distanceAbove  # Distance above the interface
        
        # Set x_up query points
        x_up = 1e-3 * np.linspace(0., 45., nOfQueryPts)
        QueryPts_up = np.stack([x_up, distance_above * np.ones(x_up.shape)], axis = 1)
        QueryPts_dn = np.stack([x_up, -distance_above * np.ones(x_up.shape)], axis = 1)
        
        # nOfNodes
        nOfNodes = len(x_up)
            
        # Check if the cases have been excuted
        if self.casesExcuted == False:
            return Observations
        
        # Loop through all Inputs
        for input_ele in input_set:
            # Open the file
            h5_file = self.frontsurf_path + "A" + string(input_ele[0]) + "_B" + string(input_ele[1]) \
                      + "_DRS" + string(input_ele[2]) + "_theta" + string(input_ele[3]) + "_fStar" + \
                      string(input_ele[4]) + "-domain.h5"
                     
            f = h5py.File(h5_file, 'r')
            
            # Get time
            times = np.array(f['time']).reshape([-1])
            times = times - np.min(times)
            nOfTSteps = times.shape[0]

            # Get coordinates
            coords = np.array(f['geometry']['vertices']) - VSstart
            Qcoords = coords @ Q.transpose()
            
            # Store the slip rates
            slip_rate_x = np.zeros([nOfTSteps, nOfNodes])
            
            # Get Slip rates
            velocity = np.array(f['vertex_fields']['velocity'])
            Qvelocity = velocity @ Q.transpose()
            
            for i in range(nOfTSteps):
                slip_rate_x[i, :] = - griddata(Qcoords, velocity[i, :, 0], QueryPts_up, method = 'cubic') \
                              + griddata(Qcoords, velocity[i, :, 0], QueryPts_dn, method = 'cubic')
            slip_rate_x = slip_rate_x.transpose()
            
            # Get the observations
            if self.fourierFlag:
                # Find the Fourier coefficients
                FourierTerms = self.FourierTerms
                T = np.max(times)

                # Compute the Fourier terms
                Ks = np.array(range(FourierTerms))
                coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * times)
                VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in slip_rate_x], 0)

                # Compute the fourier coefficients
                # print('time.shape: ', time.shape)
                observation = np.trapz(VxcoskPiTt, x=times)
                # print("observation shape: ", observation.shape)
                # Append the result from this file
                Observations.append(observation)
            
            else:
                # Find nFourier terms of values, evenly spaced in [0, T]
                T = np.max(times)
                tts = np.linspace(0., T, self.FourierTerms)
                observation_func = interp1d(times, slip_rate_x)
                observation = observation_func(tts).reshape([-1])
                Observations.append(observation)
        
        Observations = np.array(Observations)
        return Observations
    
    # Function getObservationsFrontSurf for the input_set
    def getObservationsEveryWhere(self, input_set):
        # Initialize Observations
        Observations = []
        
        # Rotation matrix Q
        alpha = 29 / 180 * np.pi
        Q = np.array([[np.cos(alpha), np.sin(alpha)], [-np.sin(alpha), np.cos(alpha)]])
        
        # VS start
        VSstart = np.array([0.006354, 0.003522])
            
        # Check if the cases have been excuted
        if self.casesExcuted == False:
            return Observations
        
        # Loop through all Inputs
        for input_ele in input_set:
            # Open the file
            h5_file = self.frontsurf_path + "A" + string(input_ele[0]) + "_B" + string(input_ele[1]) \
                      + "_DRS" + string(input_ele[2]) + "_theta" + string(input_ele[3]) + "_fStar" + \
                      string(input_ele[4]) + "-domain.h5"
                     
            f = h5py.File(h5_file, 'r')
            
            # Get time
            times = np.array(f['time']).reshape([-1])
            times = times - np.min(times)
            nOfTSteps = len(times)
            
            # Get coordinates
            coords = np.array(f['geometry']['vertices']) - VSstart
            Qcoords = coords @ Q.transpose()
            
            # Get the node list in the observable zone
            nodeList = np.all(
                np.stack(
                    [Qcoords[:, 0] >= 0., Qcoords[:, 0] <= 46.e-3, Qcoords[:, 1] >= -15.e-3, Qcoords[:, 1] <= 15.e-3]
                ), 
                axis = 0 
            )
            
            # Get Slip rates
            velocity = np.array(f['vertex_fields']['velocity'])[:, nodeList, :]
            Qvelocity = velocity @ Q.transpose()
            # print("Qvelocity shape: ", Qvelocity.shape)
            # Collapse all measurements
            slip_rate_x = Qvelocity.reshape([Qvelocity.shape[0], -1]).transpose()
            
            
            
            # Find nFourier terms of values, evenly spaced in [0, T]
            T = np.max(times)
            tts = np.linspace(0., T, len(times) // 5)
            observation_func = interp1d(times, slip_rate_x)
            observation = observation_func(tts).reshape([-1])
            Observations.append(observation)
        
        Observations = np.array(Observations)
        return Observations
    

In [ ]:
a = np.rand([5])
b = np.rand([3])
a.append(b)

# Class BayersianInv

In [198]:
# Split a hyper-rectangle into n_gridPts^K rectangles
def divideRects(low_hi, nGridPts):
    # Provide low, high values of any given rectangle, evenly divide into nGridPts
    low = low_hi[0]
    hi = low_hi[1]
    newLows = np.linspace(low, hi, nGridPts)[0 : -1].transpose()
    delta = (hi - low) / (nGridPts - 1)
    
    # Change newLows into starting points
    shits = np.meshgrid(*newLows)
    newLows = np.array([x.reshape(-1) for x in shits]).transpose()
    newHighs = newLows + delta
    areas = [np.product(delta) for i in range(len(newLows))]
    
    # Get all the  grid points
    AllGridPts = np.linspace(low, hi, nGridPts).transpose()
    
    # Change newLows into starting points
    shits = np.meshgrid(*AllGridPts)
    AllGridPts = np.array([x.reshape(-1) for x in shits]).transpose()
    AllGridPts = AllGridPts.reshape([-1, len(low)])
    
    return [[newLows[i], newHighs[i]] for i in range(len(newLows))], areas, AllGridPts

# Find the smallest rectangle that contains this point
# Rects and Rects_area has to be lists
def findSmallestRectContainsPt(Rects, Rects_area, pt):
    # Order the rectangles by area
    ord_idx = np.argsort(Rects_area)
    Rects = [Rects[i] for i in ord_idx]
    Rects_area = [Rects_area[i] for i in ord_idx]
    
#     # DEBUG LINES
#     print("Rects: ", Rects)
#     print("Rects_area: ", Rects_area)
    
    
    # Find the rectangle that pt is in
    for idx in range(len(Rects)):
        rect = Rects[idx]
#         # DEBUG LINES
#         print("idx: ", idx)
#         print("rect: ", rect)
#         print("rect[0]: ", rect[0])
#         print("rect[1]: ", rect[1])
#         print("pt: ", pt)
#         print("(np.all(rect[0] <= pt): ", (np.all(rect[0] <= pt)))
#         print("(np.all(rect[1] >= pt): ", (np.all(rect[1] >= pt)))
        
        if (np.all(rect[0] <= pt) and np.all(rect[1] >= pt)):
            res_idx = idx
            res_rect = rect
            break
    
    return res_idx, res_rect

# Substitute Rectangles in the list with a new set of rectangles
def substituteRects(Rects, Rects_area, pt, nGridPts):
    # Divide the target rectangle
    idx, rect = findSmallestRectContainsPt(Rects, Rects_area, pt)
    Rect = Rects.pop(idx)
    RectArea = Rects_area.pop(idx)
    newRects, newAreas, AllGridPts = divideRects(Rect, nGridPts)
    
    # DEBUG LINES
    print("Rect and nGridPts: ", Rect, nGridPts)
    
    # Append the new rectangles and new areas
    Rects = Rects.append(newRects)
    Rects_area = Rects_area.append(newAreas)
    
    # Print Rects
    print("Rects: ", Rects)
    print("Rects_area: ", Rects_area)
    
    # Return all new points
    return AllGridPts

In [201]:
myInv.Rects = [[np.array(myInv.u_low), np.array(myInv.u_high)]] 
myInv.Rects

[[array([0.005, 0.01 ]), array([0.015, 0.02 ])]]

Finished in 154 s!

Running case A8.8297872340425531e-03_B1.5744680851063831e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 10
Finished in 154 s!

Finished in 156 s!

Running case A1.0319148936170211e-02_B1.2127659574468085e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 9
Running case A1.4999999999999999e-02_B1.4255319148936171e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 6
Finished in 155 s!

Finished in 155 s!

Running case A5.6382978723404260e-03_B1.8297872340425531e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 19
Running case A1.3723404255319149e-02_B1.3617021276595745e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 7
Finished in 153 s!

Running case A9.8936170212765955e-03_B1.5744680851063831e-02_DRS3.0000000000000001e-06_theta6.000000000

In [ ]:
myInv.RectsArea = [np.product(myInv.u_high - myInvf.u_low)]


In [199]:
# 
substituteRects(myInv.Rects, myInv.RectsArea, myInv.maxLikelihoodUs[-1], 4)

TypeError: only integer scalar arrays can be converted to a scalar index

In [192]:
# Define Bayersian Inv that solves a Bayersian inversion problem
class BayersianInv:
    # Constructor
    def __init__(self, u_low, u_high, u, y, work_path, FourierTerms = 16, 
                 atol = 1.0e-6, si_eta = 0.5, n_samples = 20, MCMCsteps = 1000, save_path = "./", 
                 distanceAbove = 2e-3, nOfQueryPts = 10, obsFlag = 'fault', fourierFlag = False, 
                 samplesFlag = 'sampling'):
        # Set X has to be compact in R^k
        self.real_u_idx = np.array((u_low != u_high))
        self.u_low = np.array(u_low[self.real_u_idx])
        self.u_high = np.array(u_high[self.real_u_idx])
        self.dummy_u = np.array(u_low[~self.real_u_idx])
        
        
        # For each iteration, needs a region to build the grid points
        self.u_lows = [u_low]
        self.u_highs = [u_high]
        
        # Store all the Rectangles
        self.Rects = [[np.array(self.u_low), np.array(self.u_high)]] 
        self.RectsArea = [np.product(self.u_high - self.u_low)]
        
        # The true value of u, as well as the observation y to be inverted
        self.u = np.array(u)
        self.y = np.array(y)
        
        # Other parameters
        self.input_dim_All = len(u_low)
        self.input_dim = len(self.u_low)
        self.output_dim = len(self.y)
        self.work_path = work_path
        self.FourierTerms = FourierTerms
        self.atol = atol
        self.si_eta = si_eta
        self.n_samples = n_samples
        self.MCMCsteps = MCMCsteps
        self.save_path = save_path
        
        # Keep record of distanceAbove and nOfQueryPts in the field of view
        self.distanceAbove = distanceAbove
        self.nOfQueryPts = nOfQueryPts
        
        # Get where the observations are from
        self.obsFlag = obsFlag
        self.fourierFlag = fourierFlag
        self.samplesFlag = samplesFlag
        
        # Keep records of the sampled points and the corresponding observations after iteration
        self.U = np.empty([0, self.input_dim])
        self.O = np.empty([0, self.output_dim])
        self.iterations = 0
        
        # Keep record of eta, MaxMinLenRatio, GaussianProcess Emulator, average error on the sampled points, 
        # and empirical mean and stdv of the posterior after each iteration
        self.etas = []
        self.maxMinDistRatio = []
        self.GPs = []
        self.avg_errors = []
        self.mean = []
        self.stdv = []
        
        # Maximum likelihood points
        self.maxLikelihoodUs = [(self.u_low + self.u_high) / 2]
        self.maxLikelihoodUsPropL2Error = []
        self.maxLikelihoodObsPropL2Error = []
    
    # Cast u into the same length of u_low, with dummy u
    def castU(self, u_real):
        # print("u_real shape: ", u_real.shape)
        u_real_this = u_real.reshape([-1, self.real_u_idx.sum()])
        # print("u_real_this shape: ", u_real_this.shape)
        
        res = np.zeros([u_real_this.shape[0], len(self.u)])
        res[:, self.real_u_idx] = u_real_this
        res[:, ~self.real_u_idx] = self.dummy_u
        return res
    
    # Get the accumulative probability function
    def log_prob(self, u, y):
        # Apply hard constraints
        u_reshape = u.reshape([-1, self.input_dim])
        normal_idx = np.all(
            np.concatenate(
                [u_reshape >= self.u_low, u_reshape <= self.u_high], axis = 1
            ), 
            axis = 1
        )
       
        # First prior is uniform distribution
        res = np.ones(len(u_reshape))
        res[~normal_idx] = -np.inf
        
        # Add all posteriors in each iteration (since this is log of probability density)
        if (np.sum(normal_idx) > 0):
            for i in range(self.iterations):
                res[normal_idx] += -0.5 / self.etas[i] ** 2 * (
                    np.sum(
                        (y - self.GPs[i].predict(u_reshape[normal_idx])) ** 2, 
                        axis = 1
                    )
                )
        
        # Return the log_probability at the current iteration
        return res
    
    # Compute statistics: sample and calculate mean and stdv
    def compute_stats(self, n_samples, n_steps = 1_000):
#         # Sample for more points to update the empirical statistics
#         sampler = emcee.EnsembleSampler(n_samples, 
#                                         self.input_dim, 
#                                         self.log_prob, args=[y], 
#                                         vectorize = True)

#         # Initialize uniformly as the starting point
#         p0 = np.random.uniform(size = [n_samples, self.input_dim]) * (self.u_high - self.u_low) + self.u_low

#         # Get the result
#         sampler.run_mcmc(p0, n_steps)
#         samples = sampler.get_last_sample().coords
#         self.mean.append(np.mean(samples, axis = 0))
#         self.stdv.append(np.std(samples, axis = 0))
        
        # Get maximum likelihood estimate
        fun = lambda u: -self.log_prob(u, self.y)
        newCenter = opt.minimize(fun, x0 = np.mean(self.samples, axis = 0), 
                                 bounds = [(self.u_low[i], self.u_high[i]) for i in range(len(self.u_low))]
                                ).x
        
        self.maxLikelihoodUs.append(newCenter)
        
        self.maxLikelihoodUsPropL2Error.append(
            np.linalg.norm(self.u[self.real_u_idx] - self.maxLikelihoodUs[-1]) 
            / np.linalg.norm(self.u[self.real_u_idx])
        )
        self.maxLikelihoodObsPropL2Error.append(
            np.linalg.norm(self.y - self.GPs[-1].predict(self.maxLikelihoodUs[-1])) 
            / np.linalg.norm(self.y)
        )
        
        # Find the new rectangle
        
        
#         # Function to determine if a point is in the rectangles
#         def isInRectangle(setOfRects, pt):
#             res = [ for rec in setOfRects]
        
    # Get average error of a batch
    def get_avg_error_of_a_batch(self, myBatch):
        return np.mean(np.sum((myBatch.Observations - self.y) ** 2, axis = 1))
    
    # Run one iteration
    def runOneIteration(self, n_samples, n_stat_samples):
        # Switch based on samplesFlag
        if self.samplesFlag == 'sampling':
            # Sample from the current distribution for u and get the current observations
            sampler = emcee.EnsembleSampler(n_samples, 
                                            self.input_dim, 
                                            self.log_prob, args=[y], 
                                            vectorize = True)

            # Initialize uniformly as the starting point
            p0 = np.random.uniform(size = [n_samples, self.input_dim]) * (self.u_high - self.u_low) + self.u_low

            # Get the result
            sampler.run_mcmc(p0, self.MCMCsteps)
            self.samples = sampler.get_last_sample().coords

            # If there is self_etas, this is to make sure MaxMinDistRatio does not go too large
            if len(self.etas) > 0:
                eta_times = 0
                while (eta_times < 2) and (MaxMinDistRatio(self.samples) > 100.):
                    # Double the last eta
                    self.etas[-1] = 2. * self.etas[-1]
                    eta_times += 1

                    # Re-sample
                    sampler = emcee.EnsembleSampler(n_samples, 
                                                    self.input_dim, 
                                                    self.log_prob, args=[y], 
                                                    vectorize = True)

                    # Initialize uniformly as the starting point
                    p0 = np.random.uniform(size = [n_samples, self.input_dim]) * (self.u_high - self.u_low) + self.u_low

                    # Get the result
                    sampler.run_mcmc(p0, self.MCMCsteps)
                    self.samples = sampler.get_last_sample().coords
                    MCsteps = 0
                    while (MCsteps < self.MCMCsteps / 10) and (MaxMinDistRatio(self.samples) > 10.):
                        sampler.run_mcmc(sampler.get_last_sample(), 1)
                        self.samples = sampler.get_last_sample().coords
                        MCsteps += 1

                # print("eta_times, MaxMinDistRatio: ", eta_times, MaxMinDistRatio(self.samples))
        
        elif self.samplesFlag == 'grid':
            # DEBUG LINEs
            print("Before updating the rectangles")
            # print("self.samples.shape:, ", self.samples.shape)
            print("self.Rects length: ", len(self.Rects))
            print("self.RectsArea length: ", len(self.RectsArea))

            # Start with the point being maxLikelihoodUs[-1]
            self.samples = substituteRects(self.Rects, self.RectsArea, self.maxLikelihoodUs[-1], n_samples)
            
            # DEBUG LINEs
            print("After updating the rectangles")
            print("self.samples.shape:, ", self.samples.shape)
            print("self.Rects length: ", len(self.Rects))
            print("self.RectsArea length: ", len(self.RectsArea))

        # Run the forward map with the samples
        myBatch = RunABatch(self.castU(self.samples), self.work_path, FourierTerms = self.FourierTerms, 
                            distanceAbove = self.distanceAbove, nOfQueryPts = self.nOfQueryPts, 
                            obsFlag = self.obsFlag, fourierFlag = self.fourierFlag)
        self.U = np.append(self.U, self.samples, axis = 0)
        self.O = np.append(self.O, myBatch.Observations, axis = 0)

        # Get a new Gaussian process
        myGP = GP_predictor(self.samples, myBatch.Observations)
        # myGP = GP_predictor(self.U, self.O)

        # Update the recorded variables of the process
        self.GPs.append(myGP)
        self.iterations += 1
        self.etas.append(self.si_eta)
        self.avg_errors.append(self.get_avg_error_of_a_batch(myBatch))
        self.maxMinDistRatio.append(MaxMinDistRatio(self.samples))
        
        # Update stats
        self.compute_stats(n_stat_samples)
        
        # Converging flag False
        return False
    
    # Run function
    def run(self, n_iter_max = 10, n_stat_samples = 1000):
        for i in range(n_iter_max):
            # In each iteration
            print("="*30, " Iteration ", str(self.iterations), " ", "="*30)
            
            # Run one iteration
            start_time = time.time()
            converged = self.runOneIteration(self.n_samples, n_stat_samples)
            end_time = time.time()
            print("Time cost: ", str(end_time - start_time), " s")
            # print("Average error in this iteration: ", self.avg_errors[-1])
            print("Maximum to minimum distance in the sample points: ", self.maxMinDistRatio[-1])
            # print("Mean of the posterior after this iteration: ", self.mean[-1])
            # print("Stdv of the posterior after this iteration: ", self.stdv[-1])
            print("Max likelihood estimate of the posterior after this iteration: ", self.maxLikelihoodUs[-1])
            print("L2 error of u: ", self.maxLikelihoodUsPropL2Error[-1])
            print("L2 error of y: ", self.maxLikelihoodObsPropL2Error[-1])
        # Save the current model to files
        self.save()
        
    # Save the model to files
    def save(self):
        savePath = self.save_path + "models/A{0}_B{1}_DRS{2}_theta{3}_fStar{4}_nOfQPts{5}_eta{6}-{7}-Fourier-{8}-samples-{9}.pickle".format(self.u[0], self.u[1], self.u[2], self.u[3], self.u[4], self.nOfQueryPts, self.si_eta, self.obsFlag, self.fourierFlag, self.samplesFlag)
        with open(savePath, 'wb') as file:
            pickle.dump(self, file)
    
    # Plot the regression process into a gif file
    def plotGIF(self):
        plotSeries(self, self.u[self.real_u_idx], self.y, self.save_path)

        
# ===================================== Other Auxiliary functions ==========================================
# Function MaxMinDistRatio, compute the ratio of maximum to minimum distance among pts, 
# for the gaussian process to converge with a reasonable length-scaled kernel, 
# this value should not be too large
def MaxMinDistRatio(pts):
    matrix = [[np.linalg.norm(pts[i] - pts[j]) for j in range(i + 1, len(pts))] for i in range(len(pts))]
    vec = [x for y in matrix for x in y]
    return max(vec) / min(vec)


# Calculate the corresponding (to y) log likelihood of iter_step on u
def log_prob_best(self, iter_step, u, y):
    # Apply hard constraints
    u_reshape = u.reshape([-1, self.input_dim])
    normal_idx = np.all(np.concatenate([u_reshape >= self.u_low, u_reshape <= self.u_high], axis = 1), axis = 1)

    # First prior is uniform distribution
    res = np.ones(len(u_reshape))
    res[~normal_idx] = -np.inf

    # Add all posteriors in each iteration
    if (np.sum(normal_idx) > 0):
        for i in range(iter_step + 1):
            res[normal_idx] += -0.5 / self.etas[i] ** 2 * (
                np.sum(
                    (y - self.GPs[i].predict(u_reshape[normal_idx])) ** 2, 
                    axis = 1
                )
            )

    # Return the log_probability at the current iteration
    return res

# Plot the given regression process on (analytical_u, analytical_y) and save the GIF
def plotSeries(self, analytical_u, analytical_y, save_path, dpi_value = 300):
    # Calculate U_plot
    minU = self.u_low
    maxU = self.u_high
    nOfGridPoints = 100

    xis = []
    for i in range(minU.shape[0]):
        xis.append(np.linspace(minU[i], maxU[i], nOfGridPoints))

    # Generate grid and draw predictions
    UPlotGrid = np.meshgrid(xis[0], xis[1])
    UPlotGrid = np.stack(UPlotGrid, axis = 2)
    UPlotGridFat = UPlotGrid.reshape([nOfGridPoints * nOfGridPoints, maxU.shape[0]])
    
    gifName = save_path + "figures/A{0}_B{1}.gif".format(analytical_u[0], analytical_u[1])
    
    writer = imageio.get_writer(gifName, mode='I', duration = 1.0)
    
    # Maximum likelihood points
    self.maxLikelihoodUs = []
    self.maxLikelihoodUsPropL2Error = []
    self.maxLikelihoodObsPropL2Error = []
    
    # Plot the series
    for i in range(0, self.iterations):
        plt.figure(figsize = (7, 6), dpi = dpi_value)
        YPlotGridFat = log_prob_best(self, i, UPlotGridFat, analytical_y)
        YPlotGridFat = YPlotGridFat - np.max(YPlotGridFat)
        
        # Get optimized u
        idx = np.argmax(YPlotGridFat)
        
        # self.maxLikelihoodUs.append(UPlotGridFat[idx])
        
        fun = lambda u: -log_prob_best(self, i, u, analytical_y)
        newCenter = opt.minimize(fun, x0 = UPlotGridFat[idx], 
                                 bounds = [(self.u_low[i], self.u_high[i]) for i in range(len(self.u_low))]
                                ).x
        
        # DEBUG LINES
        # print("=" * 60)
        # print("i, u = ", i, newCenter)
        
        # print("-fun(u_opt), log_prob_best(u_opt): ", -fun(newCenter), log_prob_best(self, i, newCenter, analytical_y))
        # print("-fun(grid_opt), log_prob_best(grid_opt): ", -fun(UPlotGridFat[idx]), log_prob_best(self, i, UPlotGridFat[idx], analytical_y))
        self.maxLikelihoodUs.append(newCenter)
        
        self.maxLikelihoodUsPropL2Error.append(
            np.linalg.norm(analytical_u - self.maxLikelihoodUs[-1]) 
            / np.linalg.norm(analytical_u)
        )
        self.maxLikelihoodObsPropL2Error.append(
            np.linalg.norm(analytical_y - self.GPs[i].predict(self.maxLikelihoodUs[-1])) 
            / np.linalg.norm(analytical_y)
        )
        
        YPlotGrid = YPlotGridFat.reshape([nOfGridPoints, nOfGridPoints])
        cp = plt.contourf(UPlotGrid[:, :, 0], UPlotGrid[:, :, 1], np.maximum(YPlotGrid, -5.))
        
        # Give the color bar
        cbar = plt.colorbar(cp)
        plt.clim([-5., 0.])
        cbar.set_label('$-\\Phi(u)$', fontsize = 20)
        
        # Scatter the sample points
        plt.scatter(self.U[ :(i + 1) * self.n_samples, 0], 
                    self.U[ :(i + 1) * self.n_samples, 1], s = 1, color = 'white')
        plt.scatter(analytical_u[0], analytical_u[1], s = 15, color = 'red')
        plt.xlabel('$u_1$', fontsize = 20)
        plt.ylabel('$u_2$', fontsize = 20)
        plt.title("The " + str(i) + " th iteration")
        figName = save_path + "figures/shit" + str(i) + ".png"
        plt.savefig(figName, dpi = dpi_value)
        
        # Save the figures into a gif
        image = imageio.imread(figName)
        writer.append_data(image)
        !rm $figName
    writer.close()
    
    # Save a figure of L2PropError of u
    plt.figure(figsize = (7, 6), dpi = dpi_value)
    plt.scatter(range(1, 1 + len(self.maxLikelihoodUsPropL2Error)), self.maxLikelihoodUsPropL2Error, s = 20)
    plt.xlabel('Iteration', fontsize = 20)
    plt.ylabel('$\|u-u_{true}\|/\|u_{true}\|$', fontsize = 20)
    # plt.ylim([0., 0.01])
    plt.title('Relative L2 error of $u$', fontsize = 20)
    figName = save_path + "figures/A{0}_B{1}_propL2UError.png".format(analytical_u[0], analytical_u[1])
    plt.savefig(figName, dpi = dpi_value)

    # Save a figure of L2PropError of y
    plt.figure(figsize = (7, 6), dpi = dpi_value)
    plt.scatter(range(1, 1 + len(self.maxLikelihoodUsPropL2Error)), self.maxLikelihoodObsPropL2Error, s = 20)
    plt.xlabel('Iteration', fontsize = 20)
    plt.ylabel('$\|y-y_{true}\|/\|y_{true}\|$', fontsize = 20)
    # plt.ylim([0., 0.01])
    plt.title('Relative L2 error of $y$', fontsize = 20)
    figName = save_path + "figures/A{0}_B{1}_propL2YError.png".format(analytical_u[0], analytical_u[1])
    plt.savefig(figName, dpi = dpi_value)
    
    

In [193]:
# Load from file
def loadInvModel(model_path):
    with open(model_path, 'rb') as file:
        myInv = pickle.load(file)
    return myInv

# Test an inversion

In [194]:
# Test run a batch
input_set = [[0.011, 0.016, 3e-6, 0.006, 0.58], [0.012, 0.015, 2e-6, 0.003, 0.6]]
distanceAbove = 1e-3
nOfQueryPts = 10
fTerms = 32
obsFlag = 'everywhere'
samplesFlag = 'grid'
fourierFlag = True
myBatch = RunABatch(input_set, work_path, fTerms, distanceAbove, nOfQueryPts, obsFlag, fourierFlag)

In [195]:
myBatch.input_set

[(0.011, 0.016, 3e-06, 0.006, 0.58), (0.012, 0.015, 2e-06, 0.003, 0.6)]

In [196]:
# Generate test batch
u_low = np.array([0.005, 0.01, 3e-6, 0.006, 0.58])
u_high = np.array([0.015, 0.02, 3e-6, 0.006, 0.58])
nSamples = 48
siEta = 0.1
print("Test input set: ", myBatch.input_set)
print("Work path: ", work_path)

Test input set:  [(0.011, 0.016, 3e-06, 0.006, 0.58), (0.012, 0.015, 2e-06, 0.003, 0.6)]
Work path:  /home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/2d/InverseUniExp/


In [197]:
### Run an inversion test:
testCase_idx = 0
u = myBatch.input_set[testCase_idx]
y = myBatch.Observations[testCase_idx]
myInv = BayersianInv(u_low, u_high, u, y, work_path, FourierTerms = fTerms, 
                     n_samples = nSamples, 
                     si_eta = siEta, 
                     nOfQueryPts = nOfQueryPts, 
                     distanceAbove = distanceAbove, 
                     obsFlag = obsFlag, 
                     fourierFlag = fourierFlag, 
                     samplesFlag = samplesFlag
                    )
myInv.run(n_iter_max = 10)

# # Save to file
# import pickle

# #save it
# with open(f'./models/myInvA4_B6.pickle', 'wb') as file:
#     pickle.dump(myInv, file) 

==============================  Iteration  0   ==============================
Before updating the rectangles
self.Rects length:  1
self.RectsArea length:  1
Rects:  None
Rects_area:  None
After updating the rectangles
self.samples.shape:,  (2304, 2)
self.Rects length:  1
self.RectsArea length:  1
Running case A1.1382978723404255e-02_B1.8936170212765960e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 4
Running case A1.4361702127659574e-02_B1.0212765957446808e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 2
Running case A1.1808510638297871e-02_B1.5957446808510641e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 3
Running case A1.2872340425531915e-02_B1.4468085106382979e-02_DRS3.0000000000000001e-06_theta6.0000000000000001e-03_fStar5.7999999999999996e-01 on thread 5
Running case A7.5531914893617021e-03_B1.7234042553191491e-02_DRS3.0000000000000001

KeyboardInterrupt: 

In [131]:
myInv.RectsArea

[9.999999999999999e-05]

In [83]:
np.array(myInv.u)[myInv.real_u_idx]

array([0.011, 0.016])

In [ ]:
myInv.run(n_iter_max = 10)